In [6]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
from fake_useragent import UserAgent
from datetime import datetime
from tqdm.auto import tqdm

In [7]:
session = requests.session()
ua = UserAgent(verify_ssl=False)
headers = {'User-Agent': ua.random}

In [8]:
def get_reviews(url_suf, headers):
    url = 'https://www.restoran.ru' + url_suf
    req = session.get(url, headers=headers)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    rev_boxes = soup.find_all('div', {"class": "review-wrap"})
    for box in rev_boxes:
        rate_circle = box.find('div', {'class':'review-rating'})
        rating = rate_circle.find('span').text
        if rating in ('1.0', '5.0'):
            review_place = box.find('span', {'class': 'review-text-full'})
            if review_place is None:
                review_place = box.find('span', {'class': 'review-text-preview'})
            if rating == '1.0':
                bad.append(review_place.text)
            else:
                good.append(review_place.text)
    next_page = soup.find('a', {'class': 'next icon-arrow-right'})
    if next_page is not None and not next_page['href'].startswith('javascript'):
        get_reviews(next_page['href'], headers)

In [9]:
def get_nth_page(page_number, headers):
    global good, bad
    url = 'https://www.restoran.ru/msk/catalog/restaurants/all/?page=' + str(page_number)
    req = session.get(url, headers=headers)
    page = req.text
    soup = BeautifulSoup(page, 'html.parser')
    restaurants = soup.find_all('a', {'class': 'reviews-link'})
    for rest in restaurants:
        get_reviews(rest['href'], headers)

In [10]:
def run_all(n_pages, headers):
    for i in tqdm(range(n_pages)):
        get_nth_page(i+1, headers)

In [11]:
good = []
bad = []
run_all(10, headers)

In [12]:
from pymorphy2 import MorphAnalyzer
from string import punctuation
from nltk.tokenize import word_tokenize
from collections import Counter

In [14]:
def parse_revs(morph, revs):
    lemmas = []
    for text in revs:
        tokens = word_tokenize(text)
        for word in tokens:
            word = word.strip(punctuation).lower()
            if word.isalpha():
                lemmas.append(morph.parse(word)[0].normal_form)
    count = Counter(lemmas)
    count = count.most_common(int(len(count) * 0.9))
    com_words = set([word[0] for word in count])
    return com_words

In [16]:
morph = MorphAnalyzer()

In [17]:
bad_com_morph = parse_revs(morph, bad[:int(len(bad)*0.8)])
good_com_morph = parse_revs(morph, good[:int(len(good)*0.8)])

In [36]:
bad_com_chunk = chunker(bad[:int(len(bad)*0.8)])
good_com_chunk = chunker(good[:int(len(good)*0.8)])

In [37]:
dist_bad_morph = bad_com_morph - good_com_morph
dist_good_morph = good_com_morph - bad_com_morph
dist_bad_chunk = bad_com_chunk - good_com_chunk
dist_good_chunk = good_com_chunk - bad_com_chunk

In [38]:
from sklearn.metrics import accuracy_score

In [43]:
def identify_review(text):
    res = []
    segmenter = Segmenter()
    morph_vocab = MorphVocab()
    morph_tagger = NewsMorphTagger(emb)
    for text in [text]:
        doc = Doc(text)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for sent in doc.sents:
            for i in range(len(sent.tokens) - 1):
                try:
                    res.append(sent.tokens[i].lemmatize(morph_vocab).lemma)
                    if sent.tokens[i].text == 'не' and sent.tokens[i+1].pos == 'VERB' or sent.tokens[i].pos == 'ADJ' and sent.tokens[i+1].pos == 'NOUN' or sent.tokens[i].pos == 'VERB' and sent.tokens[i+1].pos == 'ADV':
                        res.append(sent.tokens[i].lemmatize(morph_vocab).lemma + ' ' + sent.tokens[i+1].lemmatize(morph_vocab).lemma)
                except Exception as e:
                    pass
            try:        
                res.append(sent.tokens[len(sent.tokens) - 1].lemmatize(morph_vocab).lemma)
            except Exception as e:
                pass
    words = set(res)
    goodness_morph = len(dist_good_morph & words)
    badness_morph = len(dist_bad_morph & words)
    goodness_chunk = len(dist_good_chunk & words)
    badness_chunk = len(dist_bad_chunk & words)
    return (goodness_morph > badness_morph, goodness_chunk > badness_chunk)

In [44]:
test_data = list(zip(bad[int(len(bad)*0.8):], [0]*int(len(bad)*0.8)))
test_data.extend(list(zip(good[int(len(bad)*0.8):], [1]*int(len(good)*0.8))))
emb = NewsEmbedding()
y_true = [r[1] for r in test_data]
y_pred = [identify_review(r[0]) for r in test_data]
y_pred_morph = [x[0] for x in y_pred]
y_pred_chunk = [x[1] for x in y_pred]
print(accuracy_score(y_true, y_pred_morph))
print(accuracy_score(y_true, y_pred_chunk))

MemoryError: Unable to allocate 95.4 MiB for an array with shape (250002, 100) and data type float32

Во-первых, из-за дисбаланса классов в обучающей выборке для оценки качества модели лучше использовать не accuracy, а F-score (или просто посмотреть точность и полноту).

Во-вторых, можно привесить словам веса в соответствии с количеством их употреблений в хороших/плохих отзывах, и учитывать это при оценке.

В-третьих, нарицательное имя или прилагательное в кавычках - признак сарказма и, соответственно, негативного отзыва, это тоже можно учесть

В-четвертых (не NLP), если на сайте продавцы/владельцы заведения могут оставлять комментарии под отзывами, то с большей вероятностью комментарии (в особенности с извинениями) будут под плохими отзывами, это тоже можно проверить при парсинге (нвличие комментариев, а также слов типа "жаль" в них).

Стоит взять не + глагол, именные группы и глагол + наречие, так как это вводит хотя бы какое-то подобие синтаксиса, важного для правильного определения тональности

In [27]:
from natasha import Doc, Segmenter, NewsMorphTagger, NewsEmbedding, MorphVocab

In [33]:
def chunker(revs):
    res = []
    segmenter = Segmenter()
    morph_vocab = MorphVocab()
    emb = NewsEmbedding()
    morph_tagger = NewsMorphTagger(emb)
    for text in revs:
        doc = Doc(text)
        doc.segment(segmenter)
        doc.tag_morph(morph_tagger)
        for sent in doc.sents:
            for i in range(len(sent.tokens) - 1):
                try:
                    res.append(sent.tokens[i].lemmatize(morph_vocab).lemma)
                    if sent.tokens[i].text == 'не' and sent.tokens[i+1].pos == 'VERB' or sent.tokens[i].pos == 'ADJ' and sent.tokens[i+1].pos == 'NOUN' or sent.tokens[i].pos == 'VERB' and sent.tokens[i+1].pos == 'ADV':
                        res.append(sent.tokens[i].lemmatize(morph_vocab).lemma + ' ' + sent.tokens[i+1].lemmatize(morph_vocab).lemma)
                except Exception as e:
                    pass
            try:
                res.append(sent.tokens[len(sent.tokens) - 1].lemmatize(morph_vocab).lemma)
            except Exception as e:
                pass
    count = Counter(res)
    count = count.most_common(int(len(count) * 0.9))
    com_words = set([word[0] for word in count])
    return com_words